<a href="https://colab.research.google.com/github/shrisha-rao/bitnet-imdb/blob/main/notebooks/bitnet_imdb_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tune a BitNet Model on IMDB
This notebook converts a small pretrained model (`bert-tiny`) into a **BitNet** with ternary weights and fine-tunes it on IMDB sentiment classification. It demonstrates:
- Custom `BitLinear` layer with weight quantization and straight-through estimator.
- Replacing all linear layers in a transformer.
- Fine-tuning with Hugging Face `Trainer`.

**Note:** Use a GPU runtime (Runtime → Change runtime type → T4 GPU) for faster training.

In [1]:
# Install dependencies
!pip install transformers datasets accelerate scikit-learn

In [2]:
!pip install --upgrade transformers

In [3]:
!pip install sentencepiece

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
)
from sklearn.metrics import accuracy_score, f1_score

## Define BitLinear Layer

In [5]:
class BitLinear(nn.Linear):
    def quantize_weights(self):
        w = self.weight
        alpha = w.abs().mean().clamp(min=1e-8)
        ternary = torch.where(w > 0.5 * alpha, alpha, torch.where(w < -0.5 * alpha, -alpha, 0.0))
        return ternary

    def forward(self, x):
        quantized_w = self.quantize_weights()
        w_ste = self.weight + (quantized_w - self.weight).detach()
        return F.linear(x, w_ste, self.bias)

## Replace Linear Layers in Model

In [6]:
def replace_linear_with_bitlinear(model):
    for name, child in model.named_children():
        if isinstance(child, nn.Linear) and name != 'classifier':
            new_layer = BitLinear(child.in_features, child.out_features, bias=child.bias is not None)
            new_layer.weight.data = child.weight.data.clone()
            if child.bias is not None:
                new_layer.bias.data = child.bias.data.clone()
            setattr(model, name, new_layer)
        else:
            replace_linear_with_bitlinear(child)

## Load Dataset and Model

In [8]:
# Load IMDB dataset (small subset for speed)
dataset = load_dataset("imdb")
train_small = dataset["train"].shuffle(seed=42).select(range(5000))
test_small = dataset["test"].shuffle(seed=42).select(range(1000))


model_name = "distilbert-base-uncased"  # 67M params, still small and fast
tokenizer = AutoTokenizer.from_pretrained(model_name)

#tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
# tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny",
#                                           trust_remote_code=True)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True,
                     max_length=512)

tokenized_train = train_small.map(tokenize_function, batched=True)
tokenized_test = test_small.map(tokenize_function, batched=True)

# Load model
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny",
#                                                            num_labels=2)
replace_linear_with_bitlinear(model)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertForSequenceClassification LOAD REPORT from: distilbert-base-uncased
Key                     | Status     | 
------------------------+------------+-
vocab_transform.bias    | UNEXPECTED | 
vocab_layer_norm.bias   | UNEXPECTED | 
vocab_layer_norm.weight | UNEXPECTED | 
vocab_transform.weight  | UNEXPECTED | 
vocab_projector.bias    | UNEXPECTED | 
pre_classifier.bias     | MISSING    | 
classifier.weight       | MISSING    | 
classifier.bias         | MISSING    | 
pre_classifier.weight   | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


## Define Metrics

In [9]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }

## Training Arguments

In [12]:
help(TrainingArguments)

Help on class TrainingArguments in module transformers.training_args:

class TrainingArguments(builtins.object)
 |  TrainingArguments(output_dir: str | None = None, per_device_train_batch_size: int = 8, num_train_epochs: float = 3.0, max_steps: int = -1, learning_rate: float = 5e-05, lr_scheduler_type: transformers.trainer_utils.SchedulerType | str = 'linear', lr_scheduler_kwargs: dict | str | None = None, warmup_steps: float = 0, optim: transformers.training_args.OptimizerNames | str = 'adamw_torch_fused', optim_args: str | None = None, weight_decay: float = 0.0, adam_beta1: float = 0.9, adam_beta2: float = 0.999, adam_epsilon: float = 1e-08, optim_target_modules: None | str | list[str] = None, gradient_accumulation_steps: int = 1, average_tokens_across_devices: bool = True, max_grad_norm: float = 1.0, label_smoothing_factor: float = 0.0, bf16: bool = False, fp16: bool = False, bf16_full_eval: bool = False, fp16_full_eval: bool = False, tf32: bool | None = None, gradient_checkpointing

In [13]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    logging_dir="./logs",
    report_to="none",
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


## Train

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


Epoch,Training Loss,Validation Loss


## Save Model

In [ ]:
model.save_pretrained("./bitnet-imdb-finetuned")
tokenizer.save_pretrained("./bitnet-imdb-finetuned")

# Zip and download (optional)
import shutil
from google.colab import files
shutil.make_archive("bitnet-imdb-finetuned", 'zip', "./bitnet-imdb-finetuned")
files.download("bitnet-imdb-finetuned.zip")

## Quick Test

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification", model="./bitnet-imdb-finetuned", tokenizer="./bitnet-imdb-finetuned")
print(classifier("This movie was absolutely wonderful!"))
print(classifier("Worst film ever made."))